In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from citipy import citipy

In [8]:
# Create 2,000 pairs of coordinates
lats = np.random.uniform(-90.000,90.000,size=2000)
lngs = np.random.uniform(-90.000,90.000,size=2000)
lats_lngs = zip(lats,lngs)
coordinates = list(lats_lngs)

In [9]:
cities = []

# Find the closest city for each pair of coordinates
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    if city not in cities:
        cities.append(city)

len(cities)

760

In [10]:
import requests
from config import weather_api_key
import time

In [11]:
# Check JSON layout
base_url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
test = requests.get(base_url + "&q=Boston").json()
test

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 93.69,
  'feels_like': 94.01,
  'temp_min': 86.14,
  'temp_max': 98.62,
  'pressure': 1009,
  'humidity': 34},
 'visibility': 10000,
 'wind': {'speed': 10.36, 'deg': 110},
 'clouds': {'all': 40},
 'dt': 1658518981,
 'sys': {'type': 2,
  'id': 2013408,
  'country': 'US',
  'sunrise': 1658482008,
  'sunset': 1658535254},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [12]:
city_data = []
set_num = 1
record_num = 1

# Begin retrieving data from OpenWeatherMap
print(
    "Beginning Data Retrieval\n",
    "-----------------------------"
)

for i, city in enumerate(cities):
    # Group data into sets of 50 to prevent time-outs.
    if i % 50 == 0 and i >= 50:
        record_num = 1
        set_num += 1
        time.sleep(60)

    print(f"Processing record {record_num} of Set {set_num} | {city}")
    record_num += 1

    city_url = base_url + "&q=" + city.replace(" ","+")

    # Parse JSON for weather data
    try:
        weather_data = requests.get(city_url).json()

        lat = weather_data["coord"]["lat"]
        lng = weather_data["coord"]["lon"]
        max_temp = weather_data["main"]["temp_max"]
        humidity = weather_data["main"]["humidity"]
        clouds = weather_data["clouds"]["all"]
        wind = weather_data["wind"]["speed"]
        country = weather_data["sys"]["country"]
        description = weather_data["weather"][0]["description"]

        city_data.append({
            "City": city.title(),
            "Country": country,
            "Lat": lat,
            "Lng": lng,
            "Max Temp": max_temp,
            "Humidity": humidity,
            "Cloudiness": clouds,
            "Wind Speed": wind,
            "Current Description": description
        })
    except:
        print(f"{city} not found. Skipping...")
        continue

# Indicate that data loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval
 -----------------------------
Processing record 1 of Set 1 | busselton
Processing record 2 of Set 1 | quatre cocos
Processing record 3 of Set 1 | edd
Processing record 4 of Set 1 | namibe
Processing record 5 of Set 1 | washington
Processing record 6 of Set 1 | baragua
Processing record 7 of Set 1 | hofn
Processing record 8 of Set 1 | same
Processing record 9 of Set 1 | pestovo
Processing record 10 of Set 1 | marinette
Processing record 11 of Set 1 | ushuaia
Processing record 12 of Set 1 | sistranda
Processing record 13 of Set 1 | oeiras do para
Processing record 14 of Set 1 | punta arenas
Processing record 15 of Set 1 | east london
Processing record 16 of Set 1 | jamestown
Processing record 17 of Set 1 | qaanaaq
Processing record 18 of Set 1 | youkounkoun
Processing record 19 of Set 1 | mahajanga
Processing record 20 of Set 1 | san nicolas
Processing record 21 of Set 1 | amderma
amderma not found. Skipping...
Processing record 22 of Set 1 | nyagan
Processing r

In [16]:
weather_df = pd.DataFrame(city_data)
weather_df['Lat'] = weather_df['Lat'].map("{:.2f}".format)
weather_df['Lng'] = weather_df['Lng'].map("{:.2f}".format)
weather_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.65,115.33,58.84,82,40,19.13,light rain
1,Quatre Cocos,MU,-20.21,57.76,70.09,77,10,16.37,light rain
2,Edd,ER,13.93,41.69,92.19,57,100,4.16,overcast clouds
3,Namibe,AO,-15.20,12.15,62.42,88,10,3.22,clear sky
4,Washington,US,47.50,-120.50,85.57,29,0,9.64,clear sky


In [17]:
weather_df.to_csv("../Weather_Database/WeatherPy_Database.csv")